In [5]:
import torch
import torchaudio
import os
import pandas as pd
import h5py
import numpy as np
from tqdm import tqdm
import glob
import gc
import json

class SimplePANN(torch.nn.Module):
    def __init__(self):
        super(SimplePANN, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.adaptive_pool = torch.nn.AdaptiveAvgPool2d((32, 32))
        self.fc = torch.nn.Linear(128 * 32 * 32, 2048)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def load_audio(audio_path, target_sr=22050, chunk_duration=1):
    waveform, sr = torchaudio.load(audio_path)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    waveform = waveform.mean(dim=0)
    
    chunk_size = chunk_duration * target_sr
    chunks = waveform.split(chunk_size)
    return chunks

def extract_pann_features(wav_chunk, pann_model, device):
    mel_spec = torchaudio.transforms.MelSpectrogram(
        sample_rate=22050,
        n_mels=64,
        n_fft=1024,
        hop_length=512
    )(wav_chunk)
    mel_spec = mel_spec.unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        chunk_features = pann_model(mel_spec)
    return chunk_features.cpu().numpy()

def extract_mel_features(wav_chunk):
    mel_spec = torchaudio.transforms.MelSpectrogram(
        sample_rate=22050,
        n_mels=64,
        n_fft=1024,
        hop_length=512
    )(wav_chunk)
    return mel_spec.numpy()

def process_audio(wav_file, df, pann_model, device, is_training=True, feature_type='pann'):
    try:
        wav_chunks = load_audio(wav_file)
        
        features = []
        for i, chunk in enumerate(wav_chunks):
            if i % 100 == 0:
                print(f"Processing chunk {i}/{len(wav_chunks)} of {wav_file}")
            if feature_type == 'pann':
                chunk_features = extract_pann_features(chunk, pann_model, device)
            else:
                chunk_features = extract_mel_features(chunk)
            features.append(chunk_features)
            
            # Clear GPU memory
            torch.cuda.empty_cache()
        
        features = np.concatenate(features, axis=0 if feature_type == 'pann' else 1)
        
        segments = []
        labels = []
        start_times = []
        end_times = []

        for i in range(len(df)):
            ith_row = df.iloc[i]
            if is_training:
                label = 0
                for col in df.columns:
                    if col not in ['Audiofilename', 'Starttime', 'Endtime']:
                        if ith_row[col] == 'POS':
                            label = 1
                            break
            else:
                label = 1 if ith_row['Q'] == 'POS' else 0

            onset = int(round(ith_row['Starttime'] * 22050 / 1024))
            offset = int(round(ith_row['Endtime'] * 22050 / 1024))
            
            if offset - onset > 0:
                if feature_type == 'mel':
                    segment = features[:, onset:offset]
                else:
                    segment = features[onset:offset]
                segments.append(segment)
                labels.append(label)
                start_times.append(ith_row['Starttime'])
                end_times.append(ith_row['Endtime'])

        return segments, labels, start_times, end_times
    except Exception as e:
        print(f"Error processing {wav_file}: {str(e)}")
        return [], [], [], []

def save_to_h5(item, filename):
    with h5py.File(filename, 'w') as hf:
        hf.create_dataset('file', data=item['file'])
        hf.create_dataset('segments', data=np.array(item['segments']))
        hf.create_dataset('labels', data=np.array(item['labels']))
        hf.create_dataset('start_times', data=np.array(item['start_times']))
        hf.create_dataset('end_times', data=np.array(item['end_times']))

def preprocess_dataset(path, pann_model, device, is_training=True, feature_type='pann'):
    subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    checkpoint_file = f'checkpoint_{feature_type}.json'
    
    # Load checkpoint if exists
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
    else:
        checkpoint = {'subdir': 0, 'file': 0}

    for subdir_idx, subdir in enumerate(subdirs[checkpoint['subdir']:], start=checkpoint['subdir']):
        subdir_path = os.path.join(path, subdir)
        csv_files = glob.glob(os.path.join(subdir_path, '*.csv'))

        for file_idx, csv_file in enumerate(csv_files[checkpoint['file']:], start=checkpoint['file']):
            df = pd.read_csv(csv_file)
            wav_file = csv_file.replace('.csv', '.wav')
            if not os.path.exists(wav_file):
                print(f"skipping {wav_file} as it does not exist.")
                continue

            print(f"Processing {wav_file}")
            try:
                segments, labels, start_times, end_times = process_audio(wav_file, df, pann_model, device, is_training, feature_type)
                if len(segments) == 0:
                    print(f"No valid segments extracted from {wav_file}")
                    continue
                
                item = {
                    'file': os.path.basename(wav_file),
                    'segments': np.array(segments),
                    'labels': np.array(labels),
                    'start_times': np.array(start_times),
                    'end_times': np.array(end_times)
                }
                
                # Save data for each file immediately
                output_file = os.path.join(path, f'{feature_type}_{os.path.basename(wav_file)}.h5')
                save_to_h5(item, output_file)
                print(f"Saved processed data to {output_file}")
                
                checkpoint['file'] = file_idx + 1
                with open(checkpoint_file, 'w') as f:
                    json.dump(checkpoint, f)
                
            except Exception as e:
                continue
            
            torch.cuda.empty_cache()
            gc.collect()

        checkpoint['file'] = 0
        checkpoint['subdir'] = subdir_idx + 1
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pann_model = SimplePANN().to(device)
    pann_model.eval()

    train_path = "/Users/zaher/Desktop/Project/Training_Set"
    val_path = "/Users/zaher/Desktop/Project/Validation_Set"

    print("Preprocessing the training data")
    preprocess_dataset(train_path, pann_model, device, is_training=True, feature_type='pann')
    preprocess_dataset(train_path, pann_model, device, is_training=True, feature_type='mel')
    print("Finished Preprocessing training data")
    print("Preprocessing validation data")
    preprocess_dataset(val_path, pann_model, device, is_training=False, feature_type='pann')
    preprocess_dataset(val_path, pann_model, device, is_training=False, feature_type='mel')
    print("Finished Preprocessing validation data")
    print("Preprocessing completed")

Preprocessing the training data
Finished Preprocessing training data
Preprocessing validation data
Processing /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 0/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 100/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 200/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 300/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 400/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 500/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 600/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 700/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 800/11185 of /Users/zaher/Desktop/Project/Validation_Set/RD/RD_06.wav
Processing chunk 900/11185 of /Users/zaher/Desktop/P

In [3]:
import os
import numpy as np
import librosa
import torch
import torchaudio
from tqdm import tqdm

class PANNFeatureExtractor:
    def __init__(self, sample_rate=22050, window_size=1024, hop_size=320, mel_bins=64, fmin=50, fmax=14000):
        self.sample_rate = sample_rate
        self.window_size = window_size
        self.hop_size = hop_size
        self.mel_bins = mel_bins
        self.fmin = fmin
        self.fmax = fmax

    def extract_features(self, audio):
        waveform = torch.from_numpy(audio).float()
        mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.sample_rate,
            n_fft=self.window_size,
            hop_length=self.hop_size,
            n_mels=self.mel_bins,
            f_min=self.fmin,
            f_max=self.fmax
        )(waveform)
        log_mel_spectrogram = torchaudio.transforms.AmplitudeToDB()(mel_spectrogram)
        return log_mel_spectrogram.numpy()

def preprocess_evaluation_dataset(data_path, output_path):
    fe = PANNFeatureExtractor()
    
    processed_files = 0
    for root, _, files in os.walk(data_path):
        for file in tqdm(files):
            if file.endswith('.wav'):
                wav_path = os.path.join(root, file)
                print(f"Processing file: {wav_path}")
                
                audio, sr = librosa.load(wav_path, sr=fe.sample_rate)
                features = fe.extract_features(audio)
                
                save_dir = output_path
                os.makedirs(save_dir, exist_ok=True)
                save_path = os.path.join(save_dir, f"{os.path.splitext(file)[0]}.npy")
                np.save(save_path, features)
                processed_files += 1

    if processed_files == 0:
        print("No wav files  found ")
    else:
        print(f"Preprocessing completed  Processed {processed_files} files.")

if __name__ == "__main__":
    EVAL_PATH = "/Users/zaher/Desktop/Project/eval_2"
    PROCESSED_EVAL_PATH = "/Users/zaher/Desktop/Project/Processed__PANN_Evaluation_Set"

    print("Preprocessing evaluation data")
    preprocess_evaluation_dataset(EVAL_PATH, PROCESSED_EVAL_PATH)
    print("Preprocessing for evaluation completed.")

Preprocessing evaluation data


  0%|                                                     | 0/6 [00:00<?, ?it/s]

Processing file: /Users/zaher/Desktop/Project/eval_2/MGE/85MGE.wav


/opt/anaconda3/lib/python3.12/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (513) may be set too low.
  warnings.warn(
 17%|███████▌                                     | 1/6 [00:00<00:03,  1.54it/s]

Processing file: /Users/zaher/Desktop/Project/eval_2/MGE/89MGE.wav


...

Processing file: /Users/zaher/Desktop/Project/eval_2/CHE23/CHE_F18.wav


 61%|██████████████████████████                 | 20/33 [00:02<00:02,  5.88it/s]

Processing file: /Users/zaher/Desktop/Project/eval_2/CHE23/CHE_F19.wav


100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 12.73it/s]

Preprocessing completed  Processed 66 files.
Preprocessing for evaluation completed.
